<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li><li><span><a href="#Финальные-комментарии-ревьюера" data-toc-modified-id="Финальные-комментарии-ревьюера-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Финальные комментарии ревьюера</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from numpy import linalg as la

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.info()
display(df.head())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Объявим класс линейной регрессии

In [3]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Посчитаем, чему равна наша метрика

In [4]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Нет, не изменится, т.к. матрица в итоге скомпенсирует себя.

**Обоснование:** 
$$
Z = XP
$$

$$
w' = (Z^T Z)^{-1} Z^T y
$$

$$
w' = ((XP)^T(XP))^{-1}(XP)^T y
$$

$$
w' = (P^T(X^TX)P)^{-1}P^TX^Ty
$$

$$
w' = (P)^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

$$
a = Zw'
$$

$$
a = XP (P)^{-1}(X^TX)^{-1}X^Ty
$$

$$
a = X(X^TX)^{-1}X^Ty
$$

Видим, что в конечном итоге, матрица, на которую мы домножили, действительно скомпенсировалась.

## Алгоритм преобразования

**Алгоритм**

1. Сгенерируем случайную матрицу размером 4х4
2. Проверим обратима ли сгенерированная матрица
3. Умножим матрицу признаков на полученную матрицу
4. Обучим на новой матрице линейную регрессию
5. Проверим значение метрики **R2** на полученных предсказаниях

**Обоснование**

Как мы доказали в предыдущем пункте, при умножении на обратимую матрицу, она в итоге скомпенсируется. Смодерилуем данный алгоритм.

In [10]:
def code(features):
    while True:
        coding_matrix = np.random.normal(size=(4,4))
        if la.det(coding_matrix) != 0:
            break            
    return features @ coding_matrix


## Проверка алгоритма

In [11]:
new_features = code(features)

model = LinearRegression()
model.fit(new_features, target)
predictions = model.predict(new_features)
print(r2_score(target, predictions))

0.4249455028666561


Как мы видим значение метрики **R2** не изменилось. Посмотрим на новую таблицу с признаками.

In [19]:
display(new_features.head())
new_features.describe()

,0,1,2,3
0,76228.312989,-31665.829772,-21033.779696,-31373.594872
1,58387.265573,-24247.873509,-16118.292646,-24062.288632
2,32264.090567,-13397.103685,-8907.881393,-13302.967676
3,64098.705503,-26634.532829,-17681.426717,-26355.327690
4,40106.450628,-16656.959787,-11069.252227,-16519.238174


,0,1,2,3
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,61347.485240,-25485.519368,-16927.104100,-25245.562989
std,15223.374427,6327.963937,4196.149819,6247.378165
min,8128.724979,-50457.190277,-33495.969678,-49928.702546
25%,51174.547979,-29761.105845,-19759.702153,-29461.805203
50%,61781.356960,-25664.705845,-17045.855114,-25414.276356
75%,71629.659858,-21257.356162,-14121.641295,-21057.397656
max,121434.703568,-3369.093877,-2251.943938,-3380.484360


Таблица стала совершенно неузнаваемая относительно исходных данных.

## Вывод

В данной задаче нам были представлены данные клиентов страховой компании.
1. Мы ответили на вопрос, можно ли матрицу признаков умножить на обратимую матрицу, чтобы качество линейной регрессии не изменилось.
2. Убедились, что можно, проверили на исходных данных.
3. Реализовали алгоритм преобразования исходных данных.
4. Проверили результат применения алгоритма по метрике **R2**. Убедились, что получили положительный результат